In [1]:
%load_ext autoreload
%autoreload 2

 # Parameter Shifting Simulation

 This notebook demonstrates how to systematically shift one or more parameters in a
 [CMR model](https://github.com/githubpsyche/jaxcmr) and then run analyses on the
 newly simulated data. We use the Healey & Kahana (2014) dataset as an example.

 **Outline**:
 1. **Parameter Setup**: Define the parameter bounds and the parameter(s) to shift.
 2. **Data & Model Loading**: Load the dataset, filter trials, import the model factory.
 3. **Simulating**: Use our custom function to re-simulate data for each parameter value.
 4. **Analyzing**: Plot various measures (SPC, CRP, PNR) for each shifted-parameter dataset.
 5. **Saving**: Save the output figures to a specified directory.

 ## 1. Parameter Setup

 Here, we specify which parameter(s) to vary, their numerical range, the run tag
 (for naming outputs), and other configuration details.

In [2]:
run_tag = "Parameter_Shifting"

model_name = "CMR"
model_factory_path = "jaxcmr.cmr.BaseCMRFactory"
fit_result_path = "fits/HealeyKahana2014_BaseCMR_best_of_3.json"

# Data and query
data_name = "HealeyKahana2014"
data_query = "data['listtype'] == -1"
data_path = "data/HealeyKahana2014.h5"
target_directory = "figures/shifting/"
connection_path = "data/peers-all-mpnet-base-v2.npy"

# Number of times to replicate the simulation per trial
experiment_count = 50

# Analysis paths
analysis_paths = [
    "jaxcmr.spc.plot_spc",
    "jaxcmr.crp.plot_crp",
    "jaxcmr.pnr.plot_pnr",
]

# Parameter bounds (example)
bounds = {
    "encoding_drift_rate": [2.220446049250313e-16, 0.9999999999999998],
    "start_drift_rate":    [2.220446049250313e-16, 0.9999999999999998],
    "recall_drift_rate":   [2.220446049250313e-16, 0.9999999999999998],
    "shared_support":      [2.220446049250313e-16, 99.9999999999999998],
    "item_support":        [2.220446049250313e-16, 9.9999999999999998],
    "learning_rate":       [2.220446049250313e-16, 0.9999999999999998],
    "primacy_scale":       [2.220446049250313e-16, 99.9999999999999998],
    "primacy_decay":       [2.220446049250313e-16, 99.9999999999999998],
    "stop_probability_scale":  [2.220446049250313e-16, 0.9999999999999998],
    "stop_probability_growth": [2.220446049250313e-16, 9.9999999999999998],
    "choice_sensitivity":  [2.220446049250313e-16, 99.9999999999999998],
    # etc.
}

## 2. Data & Model Loading

 Next, we load the original HDF5 dataset, create a trial mask for the subset
 of trials we want to analyze, and load our pre-fit parameters. We also import
 the model factory and any analysis functions we need.

In [3]:
from jaxcmr.helpers import generate_trial_mask, load_data, import_from_string, format_floats
from jaxcmr.experimental.array import to_numba_typed_dict
import os
import json
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from jaxcmr.simulation import parameter_shifted_simulate_h5_from_h5
from jax import random
from matplotlib import rcParams  # type: ignore
from tqdm import tqdm

# Load the main dataset
data = load_data(data_path)

# Create a mask selecting the trials we want
trial_mask = generate_trial_mask(data, data_query)

# Dynamically import the model factory class
model_factory = import_from_string(model_factory_path)

# Dynamically import each analysis function
analyses = [import_from_string(path) for path in analysis_paths]


# Optionally load a connectivity matrix (e.g., semantic similarity)
# For demonstration, we set up zeros if we don't want to use it
max_size = np.max(data["pres_itemnos"])
connections = jnp.zeros((max_size, max_size))
# If you wanted embeddings, you'd do something like:
# embeddings = np.load(connection_path)
# connections = compute_similarity_matrix(embeddings)

# Load pre-fit parameters
with open(fit_result_path, "r") as f:
    fit_result = json.load(f)
    # Some fit files might not contain a 'subject' field
    if "subject" not in fit_result["fits"]:
        fit_result["fits"]["subject"] = fit_result["subject"]

# Create output directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

## 3. Simulation & Plotting Loop

Here, we iterate over each parameter in `bounds`. For each parameter we:

- Create a discretized set of values.
- Simulate new datasets using the function `parameter_shifted_simulate_h5_from_h5`.
- Immediately process these simulations using our analysis functions.
- Save and display the generated figures.

By processing each parameter in turn, we avoid holding all simulation results in memory.

In [ ]:
rng = random.PRNGKey(0)

for shifted_parameter, (min_value, max_value) in tqdm(bounds.items()):

    # Define the number of parameter values using the available palette length
    color_cycle = [each["color"] for each in rcParams["axes.prop_cycle"]]
    considered_values = jnp.linspace(
        start=min_value,
        stop=max_value,
        num=len(color_cycle)
    )[:-1]  # Remove the last value for consistency

    # Simulate data for each of those parameter values
    rng, rng_iter = random.split(rng)
    sim = parameter_shifted_simulate_h5_from_h5(
        model_factory,
        data,
        connections,
        {key: jnp.array(val) for key, val in fit_result["fits"].items()},
        trial_mask,
        experiment_count,
        shifted_parameter,
        considered_values,
        rng_iter,
    )

    # Convert each dataset to a numba-friendly dictionary if needed
    # sim = [
    #     to_numba_typed_dict({key: np.array(val) for key, val in each.items()})
    #     for each in sim
    # ]

    # (Optional) Reset subjects for uniformity
    for i in range(len(sim)):
        sim[i]["subject"] *= 0

    # Create a common trial mask from the simulated data
    _trial_mask = generate_trial_mask(sim[0], data_query)

    # Loop through each analysis function
    for analysis in analyses:
        figure_str = (
            f"{model_name}_{shifted_parameter.title()}_{run_tag}_"
            f"{analysis.__name__[5:]}_{data_name}.png"
        )
        print(f"Saving figure: {target_directory}{figure_str}")

        # Create a color cycle using a continuous colormap
        cmap = plt.get_cmap("viridis")
        color_cycle = [cmap(i) for i in np.linspace(0, 1, len(considered_values))]
        color_cycle = [mcolors.rgb2hex(c) for c in color_cycle]

        # Run the analysis function on the simulated datasets
        axis = analysis(
            datasets=sim,
            trial_masks=[_trial_mask] * len(considered_values),
            color_cycle=color_cycle,
            distances=1 - connections,  # optional; adjust as needed
            axis=plt.gca(),
            labels=format_floats(considered_values, 1),
            contrast_name=shifted_parameter,
            # handle_repetitions=handle_repetitions[i],
        )

        # Format the plot (font sizes, legend location, etc.)
        axis.tick_params(labelsize=14)
        axis.set_xlabel(axis.get_xlabel(), fontsize=16)
        axis.set_ylabel(axis.get_ylabel(), fontsize=16)
        axis.legend(loc="upper left", bbox_to_anchor=(1, 1))

        # Save and display the figure
        plt.savefig(f"{target_directory}{figure_str}", bbox_inches="tight", dpi=600)
        plt.show()